# Vanderbeis Local Warming Model

For this to solve, you'll need to install the Clp package. Canvas has directions for installing packages


In [ ]:
using JuMP
using Clp
using GamsStructure
using NamedArrays

Load the data

In [2]:
GU = load_universe("julia_data")

Sets

d => Dates for which we have temperature reports

Parameters

ave => (:d,) => Average temperatures
day => (:d,) => Assignment of dates to days


Build and optimize the model. This is taking a long time and I'm not certain why. (julia version 1.8)

In [3]:
m = Model(Clp.Optimizer)

D = [d for d∈GU[:d]]

day = GU[:day]
ave = GU[:ave]

@variables(m,begin
    DEV[GU[:d]] >= 0 #Deviation
    T[GU[:d]]
    X0
    X1
    X2
    X3
    X4
    X5
end)

@objective(m,Min, sum(DEV[d] for d∈GU[:d]))

@constraint(m,tdef[d∈GU[:d]],
    T[d] == X0 + X1*day[[d]] + X2*cos(2*pi*day[[d]]/365.25)
                             + X3*sin(2*pi*day[[d]]/365.25)
                             + X4*cos(2*pi*day[[d]]/(10.7*365.25))
                             + X5*sin(2*pi*day[[d]]/(10.7*365.25))
)

@constraint(m,devlb[d∈GU[:d]],
    DEV[d] >= ave[[d]] - T[d]
)

@constraint(m,devub[d∈GU[:d]],
    DEV[d] >= T[d] - ave[[d]]
)

optimize!(m)

Get the last 5 variables and display them.

In [ ]:
for var in all_variables(m)[end-5:end]
    println("$var => $(value(var))")
end

In [ ]:
println("Warming = $(value(m[:X1])*365.25*100)")

In [ ]:
D = [d for d∈GU[:d]]
day = NamedArray(GU[:day][:d],D)
ave = NamedArray(GU[:ave][:d],D)


m = Model(Clp.Optimizer)


@variables(m,begin
    DEV[D] >= 0 #Deviation
    T[D]
    X0
    X1
    X2
    X3
    X4
    X5
end)

@objective(m,Min, sum(DEV[d] for d∈D))

@constraint(m,tdef[d∈D],
    T[d] == X0 + X1*day[[d]] + X2*cos(2*pi*day[[d]]/365.25)
                             + X3*sin(2*pi*day[[d]]/365.25)
                             + X4*cos(2*pi*day[[d]]/(10.7*365.25))
                             + X5*sin(2*pi*day[[d]]/(10.7*365.25))
)

@constraint(m,devlb[d∈D],
    DEV[d] >= ave[[d]] - T[d]
)

@constraint(m,devub[d∈D],
    DEV[d] >= T[d] - ave[[d]]
)

optimize!(m)
